<a href="https://colab.research.google.com/github/harshy105/Uni_luebeck/blob/main/ML_exercise_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCA, ICA and Sparse Coding

## Preamble
The following code downloads and imports all necessary files and modules into the virtual machine of Colab. Please make sure to execute it before solving this exercise.

In [ ]:
import sys, os
if 'google.colab' in sys.modules:
  if os.getcwd() == '/content':
    !git clone 'https://github.com/inb-uni-luebeck/cs5450.git'
    os.chdir('cs5450')

## Applying unsupervised methods


Import and helper functions. 

In [ ]:
import numpy as np
import math
import random
from sklearn.decomposition import PCA, FastICA, DictionaryLearning
from matplotlib import pyplot as plt
%matplotlib inline

def random_patches(data, img_height, img_width, n_patches, patch_size):
    # RANDOMPATCHES draw random square-shaped patches from the data.
    #
    # INPUT:
    #   data : ndarray containing the data (i.e. mnist or olivetti)
    #   img_height : height of the images
    #   img_width : width of the images
    #   n_patches : number of patches to be drawn
    #   patch_size : size (both width and height) of the square-shaped patches
    #
    # OUTPUT:
    #   patches : random patches

    # reshape data 2d -> 3d
    n_images = data.shape[0]
    data = data.reshape(n_images, img_height, img_width)

    # extract random patches
    patches = []#np.zeros((n_patches, patch_size*patch_size))
    for i in range(n_patches):
        patch_top = random.randint(0, img_height - patch_size)
        patch_left = random.randint(0, img_width - patch_size)
        p = data[random.randint(0, n_images-1), patch_top:patch_top+patch_size, patch_left:patch_left+patch_size]
        patches.append(p)
        #patches[i,:] = p[:]
        
    patches = np.stack(patches)
    patches = patches.reshape(patches.shape[0], -1) #flatten each patch
        
    return patches

def show_in_grid(images, height, width):
    # flatten images if necessary
    images = images.reshape(images.shape[0], -1)
    
    # normalize patches
    images = images / (np.abs(images).max(axis=1, keepdims=True)+1e-6)
    
    # reshape into images
    images = images.reshape(-1, height, width)
    
    # make images fit into a rectangular area
    grid_width = math.ceil(math.sqrt(images.shape[0]))
    grid_height = math.ceil(images.shape[0] / grid_width)
    
    empty_cells = grid_width * grid_height - images.shape[0]
    
    # fill empty cells
    if empty_cells > 0:
        padding = np.zeros((empty_cells, height, width))
        images = np.concatenate((images, padding))
        
    # rearrange basis into grid and also switch width and height so x and y axis are not switched
    images = images.reshape(grid_height, grid_width, height, width)
    images = images.transpose(0, 3, 1, 2)
    
    plt.figure(figsize = (10,10))
    plt.imshow(images.reshape(grid_height*height, grid_width*width), cmap='gray')
    plt.show()

In [ ]:
## Define variables

#  dataset : relative path to the .npz file containing the data
dataset = 'data/mnist.npz' #'data/olivetti.npz'

#  n_patches : number of random patches to extract from the images
n_patches = 10000
#  patch_size : size (both width and height) of the square-shaped patches
patch_size = 25

#  n_basis : number of new basis vectors
n_basis = 100

#  method : string describing whether to perform pca, ica, or sc
method = 'sc'

#  n_iterations : number of iterations (only for sparse coding)
n_iterations = 50

In [ ]:
## Compute and show the principal components

#  load dataset
given_dataset = np.load(dataset)
data = given_dataset['data']

#  convert data to floating point (double precision)
data = data.astype('double')
# print(data)

#  get number of images (observations) and number of pixels (features)
n_images, n_pixels = data.shape
# print(data.shape)

#  get height and width of the images
img_height = int(np.sqrt(n_pixels))
img_width = img_height
assert isinstance(img_height, int) and isinstance(img_width, int), "Image dimensions need to be integers."

#  enforce mean-free data vectors
mean_free_data = data - np.mean(data, axis=0)
# print(mean_free_data)

#  draw random patches from data.
patches = random_patches(mean_free_data, img_height, img_width, n_patches, patch_size)

#  perform the chosen unsupervised method on the drawn patches
if method == 'pca':
    pca = PCA(n_basis)
    basis = pca.fit(patches).components_
elif method == 'ica':
    ica = FastICA(n_basis)
    basis = ica.fit(patches).components_
elif method == 'sc':
    sc = DictionaryLearning(n_basis, n_jobs=-1, verbose=True)
    basis = sc.fit_transform(patches)

#  show the new set of basis vectors (only the first n_basis)
show_in_grid(basis[:n_basis], patch_size, patch_size)

In [ ]:
## Use different subsets of the new basis vectors to reconstruct a random patch
from sklearn.decomposition import sparse_encode

#  draw a random patch
random_patch = random_patches(mean_free_data, img_height, img_width, 1, patch_size)
# print(random_patch)

#  reconstruct the random patch
reconstructions = []
for i in range(n_basis):
    sub_basis = basis[0:i+1]
    
    if method == 'sc':
      sparse_signal = sparse_encode(random_patch, sub_basis)
      part = np.matmul(sparse_signal, sub_basis)
      reconstructions.append(part)
    else:
        proj = np.matmul(np.linalg.pinv(sub_basis), np.matmul(sub_basis, np.transpose(random_patch)))
        reconstructions.append(proj)
    
reconstructions = np.stack(reconstructions)

#  show the reconstructed patch
show_in_grid(reconstructions, patch_size, patch_size)

whitened data.

In [ ]:
#%  epsilon : avoid negative numbers while computing stdDeviations
epsilon = 0.1

#  perform principal component analysis (help pca)
basis = -1 #get basis vector components

#  compute the covariance matrix of the patches
covariance = -1

#  compute diagonal matrix S containing the standard deviations (c.f. page 6)
std_deviations = -1

#  pca whitening (c.f. equation 1.3)
xPCAwhite = -1

#  zca whitening
xZCAwhite = -1

#  visualize the whitened data
# show_in_grid(xZCAwhite[:n_basis], patch_size, patch_size)